# Concierge Agents System

A comprehensive multi-agent orchestration platform built on Google's Agent Development Kit (ADK).

This notebook demonstrates:
- Agent orchestration patterns (parallel, sequential, loop)
- Tool integration (MCP, custom, built-in, OpenAPI)
- Long-running operations with pause/resume
- Session management and long-term memory
- Context engineering and compaction
- Comprehensive observability
- Agent evaluation and A2A protocol
- Deployment capabilities

## Setup

Install required dependencies and configure the environment.

In [ ]:
# Install required dependencies
!pip install google-genai requests openapi-spec-validator aiohttp prometheus-client structlog pydantic python-dotenv

# Note: Using google-genai as the LLM provider.
# Google ADK (Agent Development Kit) is referenced in requirements but we're using
# google.generativeai SDK which provides agent-like capabilities through the Gemini API.

### Environment Configuration

Set up API keys and environment variables. Create a `.env` file in the same directory with your API keys:

```
GOOGLE_API_KEY=your_google_api_key_here
GOOGLE_SEARCH_API_KEY=your_search_api_key_here
GOOGLE_SEARCH_ENGINE_ID=your_search_engine_id_here
```

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify API keys are set (optional - for development)
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
if not GOOGLE_API_KEY:
    print("Warning: GOOGLE_API_KEY not set. Please configure your .env file.")
else:
    print("✓ Environment configured successfully")

### Imports and Initial Configuration

In [ ]:
# Core imports
import asyncio
import json
import sqlite3
from abc import ABC, abstractmethod
from dataclasses import dataclass, field, asdict
from datetime import datetime
from typing import Any, Callable, Dict, List, Optional, Type
from enum import Enum
import uuid
import time

# Google ADK imports
import google.generativeai as genai

# HTTP and API imports
import requests
import aiohttp
from openapi_spec_validator import validate_spec
from openapi_spec_validator.readers import read_from_filename

# Observability imports
import structlog
from prometheus_client import Counter, Histogram, Gauge

# Utilities
from pydantic import BaseModel, Field, validator

# Configure Google Generative AI
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)

# Configure structured logging
structlog.configure(
    processors=[
        structlog.processors.TimeStamper(fmt="iso"),
        structlog.processors.JSONRenderer()
    ]
)

print("✓ All imports loaded successfully")

## Core Components

Implementation of the Agent Core Engine and fundamental data models.

### Core Data Models

Define the fundamental data structures used throughout the system.

In [ ]:
@dataclass
class Task:
    """Represents a task for an agent to execute."""
    task_id: str
    description: str
    input_data: Dict[str, Any]
    context: Optional[Dict[str, Any]] = None
    metadata: Optional[Dict[str, Any]] = None
    
    def __post_init__(self):
        if self.context is None:
            self.context = {}
        if self.metadata is None:
            self.metadata = {}


@dataclass
class ToolResult:
    """Result from tool execution."""
    success: bool
    data: Any
    error: Optional[str] = None
    metadata: Dict[str, Any] = field(default_factory=dict)


@dataclass
class ToolCall:
    """Record of a tool invocation."""
    tool_name: str
    params: Dict[str, Any]
    result: ToolResult
    execution_time: float


@dataclass
class AgentResponse:
    """Response from agent execution."""
    task_id: str
    agent_id: str
    output: Any
    tool_calls: List[ToolCall]
    execution_time: float
    status: str  # 'success', 'error', 'partial'
    error: Optional[Exception] = None


@dataclass
class Message:
    """Conversation message for history tracking."""
    role: str  # 'user', 'assistant', 'system'
    content: str
    timestamp: datetime
    metadata: Dict[str, Any] = field(default_factory=dict)


@dataclass
class AgentState:
    """Serializable agent state for persistence."""
    agent_id: str
    conversation_history: List[Message]
    tool_states: Dict[str, Any]
    context: Dict[str, Any]
    timestamp: datetime
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert state to dictionary for serialization."""
        return {
            'agent_id': self.agent_id,
            'conversation_history': [
                {
                    'role': msg.role,
                    'content': msg.content,
                    'timestamp': msg.timestamp.isoformat(),
                    'metadata': msg.metadata
                }
                for msg in self.conversation_history
            ],
            'tool_states': self.tool_states,
            'context': self.context,
            'timestamp': self.timestamp.isoformat()
        }
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'AgentState':
        """Create state from dictionary."""
        return cls(
            agent_id=data['agent_id'],
            conversation_history=[
                Message(
                    role=msg['role'],
                    content=msg['content'],
                    timestamp=datetime.fromisoformat(msg['timestamp']),
                    metadata=msg.get('metadata', {})
                )
                for msg in data['conversation_history']
            ],
            tool_states=data['tool_states'],
            context=data['context'],
            timestamp=datetime.fromisoformat(data['timestamp'])
        )


print("✓ Core data models defined successfully")

### LLM Configuration and Agent Core Engine

Implementation of the ConciergeAgent class using Google's Generative AI.

In [ ]:
@dataclass
class LLMConfig:
    """Configuration for LLM."""
    provider: str  # e.g., 'gemini', 'openai'
    model: str
    temperature: float = 0.7
    max_tokens: int = 2048
    api_key: Optional[str] = None
    
    def __post_init__(self):
        if self.api_key is None:
            self.api_key = os.getenv('GOOGLE_API_KEY')


class ConciergeAgent:
    """Wrapper around Google Generative AI with enhanced capabilities."""
    
    def __init__(self, name: str, llm_config: LLMConfig, tools: Optional[List[Any]] = None):
        """
        Initialize a ConciergeAgent.
        
        Args:
            name: Unique identifier for the agent
            llm_config: Configuration for the LLM
            tools: Optional list of tools available to the agent
        """
        self.agent_id = name
        self.llm_config = llm_config
        self.tools: Dict[str, Any] = {}
        self.conversation_history: List[Message] = []
        self.context: Dict[str, Any] = {}
        
        # Initialize Google Generative AI model
        if llm_config.provider == 'gemini':
            generation_config = {
                'temperature': llm_config.temperature,
                'max_output_tokens': llm_config.max_tokens,
            }
            self.model = genai.GenerativeModel(
                model_name=llm_config.model,
                generation_config=generation_config
            )
            self.chat = self.model.start_chat(history=[])
        else:
            raise ValueError(f"Unsupported LLM provider: {llm_config.provider}")
        
        # Register tools if provided
        if tools:
            for tool in tools:
                self.add_tool(tool)
    
    def add_tool(self, tool: Any) -> None:
        """
        Register a tool with the agent.
        
        Args:
            tool: Tool instance to register
        """
        tool_name = getattr(tool, 'name', tool.__class__.__name__)
        self.tools[tool_name] = tool
    
    def execute(self, task: Task, session: Optional[Any] = None) -> AgentResponse:
        """
        Execute a task using the agent.
        
        Args:
            task: Task to execute
            session: Optional session for state management
            
        Returns:
            AgentResponse containing execution results
        """
        start_time = time.time()
        tool_calls: List[ToolCall] = []
        
        try:
            # Add task context to conversation history
            user_message = Message(
                role='user',
                content=task.description,
                timestamp=datetime.now(),
                metadata={'task_id': task.task_id, 'input_data': task.input_data}
            )
            self.conversation_history.append(user_message)
            
            # Prepare prompt with context
            prompt = task.description
            if task.input_data:
                prompt += f"\n\nInput Data: {json.dumps(task.input_data, indent=2)}"
            if task.context:
                prompt += f"\n\nContext: {json.dumps(task.context, indent=2)}"
            
            # Execute with LLM
            response = self.chat.send_message(prompt)
            output = response.text
            
            # Add response to conversation history
            assistant_message = Message(
                role='assistant',
                content=output,
                timestamp=datetime.now(),
                metadata={'task_id': task.task_id}
            )
            self.conversation_history.append(assistant_message)
            
            execution_time = time.time() - start_time
            
            return AgentResponse(
                task_id=task.task_id,
                agent_id=self.agent_id,
                output=output,
                tool_calls=tool_calls,
                execution_time=execution_time,
                status='success'
            )
            
        except Exception as e:
            execution_time = time.time() - start_time
            return AgentResponse(
                task_id=task.task_id,
                agent_id=self.agent_id,
                output=None,
                tool_calls=tool_calls,
                execution_time=execution_time,
                status='error',
                error=e
            )
    
    def get_state(self) -> AgentState:
        """
        Get the current state of the agent.
        
        Returns:
            AgentState containing serializable state
        """
        return AgentState(
            agent_id=self.agent_id,
            conversation_history=self.conversation_history.copy(),
            tool_states={name: getattr(tool, 'state', {}) for name, tool in self.tools.items()},
            context=self.context.copy(),
            timestamp=datetime.now()
        )
    
    def restore_state(self, state: AgentState) -> None:
        """
        Restore agent state from a saved state.
        
        Args:
            state: AgentState to restore
        """
        self.conversation_history = state.conversation_history.copy()
        self.context = state.context.copy()
        
        # Restore tool states
        for tool_name, tool_state in state.tool_states.items():
            if tool_name in self.tools:
                tool = self.tools[tool_name]
                if hasattr(tool, 'restore_state'):
                    tool.restore_state(tool_state)
        
        # Recreate chat with history
        if self.llm_config.provider == 'gemini':
            # Convert conversation history to Gemini format
            history = []
            for msg in self.conversation_history:
                if msg.role == 'user':
                    history.append({'role': 'user', 'parts': [msg.content]})
                elif msg.role == 'assistant':
                    history.append({'role': 'model', 'parts': [msg.content]})
            
            self.chat = self.model.start_chat(history=history)


print("✓ ConciergeAgent class implemented successfully")

### Example: Agent Creation and Basic Execution

Demonstrate creating an agent and executing a simple task.

In [ ]:
# Create LLM configuration
llm_config = LLMConfig(
    provider='gemini',
    model='gemini-pro',
    temperature=0.7,
    max_tokens=1024
)

# Create a ConciergeAgent
agent = ConciergeAgent(
    name='research-assistant',
    llm_config=llm_config
)

print(f"✓ Created agent: {agent.agent_id}")
print(f"  LLM Provider: {agent.llm_config.provider}")
print(f"  Model: {agent.llm_config.model}")

In [ ]:
# Create a simple task
task = Task(
    task_id=str(uuid.uuid4()),
    description="Explain what a multi-agent system is in 2-3 sentences.",
    input_data={},
    context={'domain': 'artificial intelligence'},
    metadata={'priority': 'high'}
)

print(f"✓ Created task: {task.task_id}")
print(f"  Description: {task.description}")

In [ ]:
# Execute the task
response = agent.execute(task)

print(f"\n{'='*60}")
print(f"AGENT RESPONSE")
print(f"{'='*60}")
print(f"Task ID: {response.task_id}")
print(f"Agent ID: {response.agent_id}")
print(f"Status: {response.status}")
print(f"Execution Time: {response.execution_time:.3f}s")
print(f"\nOutput:\n{response.output}")
print(f"\nTool Calls: {len(response.tool_calls)}")
if response.error:
    print(f"Error: {response.error}")

In [ ]:
# Retrieve agent state
state = agent.get_state()

print(f"\n{'='*60}")
print(f"AGENT STATE")
print(f"{'='*60}")
print(f"Agent ID: {state.agent_id}")
print(f"Timestamp: {state.timestamp}")
print(f"Conversation History: {len(state.conversation_history)} messages")
print(f"Registered Tools: {len(state.tool_states)}")
print(f"\nConversation History:")
for i, msg in enumerate(state.conversation_history, 1):
    print(f"  {i}. [{msg.role}] {msg.content[:80]}..." if len(msg.content) > 80 else f"  {i}. [{msg.role}] {msg.content}")

In [ ]:
# Demonstrate state serialization
state_dict = state.to_dict()
print("\n✓ State serialized to dictionary")
print(f"  Keys: {list(state_dict.keys())}")

# Demonstrate state restoration
restored_state = AgentState.from_dict(state_dict)
print("\n✓ State restored from dictionary")
print(f"  Agent ID: {restored_state.agent_id}")
print(f"  Messages: {len(restored_state.conversation_history)}")

## Orchestration

Implementation of parallel, sequential, and loop execution patterns.

### Parallel Executor

Execute multiple agents concurrently using asyncio.

In [ ]:
@dataclass
class AggregatedResult:
    """Aggregated results from multiple agent executions."""
    responses: List[AgentResponse]
    successful_count: int
    failed_count: int
    total_execution_time: float
    errors: List[Exception]


class ParallelExecutor:
    """Executes multiple agents concurrently."""
    
    def __init__(self, agents: List[ConciergeAgent]):
        """
        Initialize parallel executor.
        
        Args:
            agents: List of agents to execute in parallel
        """
        self.agents = agents
    
    async def execute_all(self, tasks: List[Task]) -> List[AgentResponse]:
        """
        Execute all agents concurrently with their respective tasks.
        
        Args:
            tasks: List of tasks (one per agent)
            
        Returns:
            List of agent responses
        """
        if len(tasks) != len(self.agents):
            raise ValueError(f"Number of tasks ({len(tasks)}) must match number of agents ({len(self.agents)})")
        
        # Create async tasks for each agent
        async_tasks = []
        for agent, task in zip(self.agents, tasks):
            async_tasks.append(self._execute_agent_async(agent, task))
        
        # Execute all tasks concurrently
        responses = await asyncio.gather(*async_tasks, return_exceptions=True)
        
        # Convert exceptions to error responses
        processed_responses = []
        for i, response in enumerate(responses):
            if isinstance(response, Exception):
                # Create error response
                processed_responses.append(AgentResponse(
                    task_id=tasks[i].task_id,
                    agent_id=self.agents[i].agent_id,
                    output=None,
                    tool_calls=[],
                    execution_time=0.0,
                    status='error',
                    error=response
                ))
            else:
                processed_responses.append(response)
        
        return processed_responses
    
    async def _execute_agent_async(self, agent: ConciergeAgent, task: Task) -> AgentResponse:
        """Execute a single agent asynchronously."""
        # Run the synchronous execute method in a thread pool
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(None, agent.execute, task)
    
    def aggregate_results(self, responses: List[AgentResponse]) -> AggregatedResult:
        """
        Aggregate results from multiple agent executions.
        
        Args:
            responses: List of agent responses
            
        Returns:
            Aggregated result summary
        """
        successful = [r for r in responses if r.status == 'success']
        failed = [r for r in responses if r.status == 'error']
        errors = [r.error for r in failed if r.error is not None]
        total_time = sum(r.execution_time for r in responses)
        
        return AggregatedResult(
            responses=responses,
            successful_count=len(successful),
            failed_count=len(failed),
            total_execution_time=total_time,
            errors=errors
        )


print("✓ ParallelExecutor class implemented successfully")

### Example: Parallel Agent Execution

Demonstrate multiple agents running concurrently.

In [ ]:
# Create multiple agents for parallel execution
agent1 = ConciergeAgent(
    name='summarizer',
    llm_config=llm_config
)

agent2 = ConciergeAgent(
    name='analyzer',
    llm_config=llm_config
)

agent3 = ConciergeAgent(
    name='translator',
    llm_config=llm_config
)

print(f"✓ Created 3 agents for parallel execution")
print(f"  Agent 1: {agent1.agent_id}")
print(f"  Agent 2: {agent2.agent_id}")
print(f"  Agent 3: {agent3.agent_id}")

In [ ]:
# Create tasks for each agent
task1 = Task(
    task_id=str(uuid.uuid4()),
    description="Summarize the key benefits of multi-agent systems in one sentence.",
    input_data={}
)

task2 = Task(
    task_id=str(uuid.uuid4()),
    description="Analyze the main challenges in building multi-agent systems.",
    input_data={}
)

task3 = Task(
    task_id=str(uuid.uuid4()),
    description="Explain what agent orchestration means in simple terms.",
    input_data={}
)

# Create parallel executor and run agents
parallel_executor = ParallelExecutor([agent1, agent2, agent3])

print(f"\n{'='*60}")
print(f"PARALLEL EXECUTION")
print(f"{'='*60}\n")
print(f"Executing {len(parallel_executor.agents)} agents in parallel...\n")

# Execute all agents concurrently
responses = await parallel_executor.execute_all([task1, task2, task3])

# Display results
for i, response in enumerate(responses, 1):
    print(f"Agent {i}: {response.agent_id}")
    print(f"  Status: {response.status}")
    print(f"  Execution Time: {response.execution_time:.3f}s")
    if response.status == 'success':
        output_preview = response.output[:100] + '...' if len(response.output) > 100 else response.output
        print(f"  Output: {output_preview}")
    else:
        print(f"  Error: {response.error}")
    print()

# Aggregate results
aggregated = parallel_executor.aggregate_results(responses)

print(f"\n{'='*60}")
print(f"AGGREGATED RESULTS")
print(f"{'='*60}")
print(f"Total Agents: {len(responses)}")
print(f"Successful: {aggregated.successful_count}")
print(f"Failed: {aggregated.failed_count}")
print(f"Total Execution Time: {aggregated.total_execution_time:.3f}s")
if aggregated.errors:
    print(f"\nErrors:")
    for error in aggregated.errors:
        print(f"  - {error}")

### Sequential Executor

Execute agents in order, passing outputs as inputs to the next agent.

In [ ]:
class SequentialExecutor:
    """Executes agents in order, passing outputs as inputs."""
    
    def __init__(self, agents: List[ConciergeAgent]):
        """
        Initialize sequential executor.
        
        Args:
            agents: List of agents to execute sequentially
        """
        self.agents = agents
    
    def execute_chain(self, initial_task: Task) -> AgentResponse:
        """
        Execute agents in sequence, passing output to next agent.
        
        Args:
            initial_task: Initial task for the first agent
            
        Returns:
            Final agent response from the last agent in the chain
        """
        current_task = initial_task
        responses = []
        
        for i, agent in enumerate(self.agents):
            # Execute current agent
            response = agent.execute(current_task)
            responses.append(response)
            
            # Check for failure
            if response.status == 'error':
                # Halt the chain on error
                print(f"Chain halted at agent {i+1}/{len(self.agents)} due to error: {response.error}")
                return response
            
            # Prepare task for next agent (if not last agent)
            if i < len(self.agents) - 1:
                current_task = self._pass_output_to_next(response, self.agents[i + 1])
        
        # Return the final response
        return responses[-1]
    
    def _pass_output_to_next(self, output: AgentResponse, next_agent: ConciergeAgent) -> Task:
        """
        Transform output from one agent into input for the next.
        
        Args:
            output: Response from previous agent
            next_agent: Next agent in the chain
            
        Returns:
            New task for the next agent
        """
        # Create new task with previous output as input
        return Task(
            task_id=str(uuid.uuid4()),
            description=f"Process the following input from the previous agent: {output.output}",
            input_data={
                'previous_output': output.output,
                'previous_agent': output.agent_id,
                'previous_task_id': output.task_id
            },
            context={
                'chain_position': len([a for a in self.agents if a.agent_id != next_agent.agent_id]) + 1,
                'total_agents': len(self.agents)
            }
        )


print("✓ SequentialExecutor class implemented successfully")

### Example: Sequential Agent Execution

Demonstrate agents executing in a chain, with each agent processing the previous agent's output.

In [ ]:
# Create agents for sequential execution
researcher = ConciergeAgent(
    name='researcher',
    llm_config=llm_config
)

summarizer = ConciergeAgent(
    name='summarizer',
    llm_config=llm_config
)

reviewer = ConciergeAgent(
    name='reviewer',
    llm_config=llm_config
)

# Create initial task
initial_task = Task(
    task_id=str(uuid.uuid4()),
    description="Research and explain the concept of agent orchestration in AI systems.",
    input_data={'topic': 'agent orchestration'},
    context={'domain': 'artificial intelligence'}
)

# Create sequential executor and run chain
sequential_executor = SequentialExecutor([researcher, summarizer, reviewer])

print(f"\n{'='*60}")
print(f"SEQUENTIAL EXECUTION")
print(f"{'='*60}\n")
print(f"Executing {len(sequential_executor.agents)} agents in sequence...\n")

# Execute the chain
final_response = sequential_executor.execute_chain(initial_task)

print(f"\n{'='*60}")
print(f"FINAL RESULT")
print(f"{'='*60}")
print(f"Final Agent: {final_response.agent_id}")
print(f"Status: {final_response.status}")
print(f"Execution Time: {final_response.execution_time:.3f}s")
print(f"\nFinal Output:\n{final_response.output}")

### Loop Executor

Execute an agent repeatedly based on a condition, with iteration state management.

In [ ]:
class LoopExecutor:
    """Executes agent repeatedly based on condition."""
    
    def __init__(self, agent: ConciergeAgent, condition: Callable[[AgentResponse], bool], max_iterations: int = 100):
        """
        Initialize loop executor.
        
        Args:
            agent: Agent to execute in loop
            condition: Function that returns True to continue loop, False to stop
            max_iterations: Maximum number of iterations to prevent infinite loops
        """
        self.agent = agent
        self.condition = condition
        self.max_iterations = max_iterations
        self.iteration_state: Dict[str, Any] = {}
    
    def execute_loop(self, initial_task: Task) -> List[AgentResponse]:
        """
        Execute agent in a loop until condition is met or max iterations reached.
        
        Args:
            initial_task: Initial task for the first iteration
            
        Returns:
            List of agent responses from all iterations
        """
        responses = []
        current_task = initial_task
        iteration = 0
        
        while iteration < self.max_iterations:
            iteration += 1
            
            # Update iteration state
            self.iteration_state['current_iteration'] = iteration
            self.iteration_state['max_iterations'] = self.max_iterations
            
            # Add iteration info to task context
            current_task.context = current_task.context or {}
            current_task.context['iteration'] = iteration
            current_task.context['max_iterations'] = self.max_iterations
            
            # Execute agent
            response = self.agent.execute(current_task)
            responses.append(response)
            
            # Check for error
            if response.status == 'error':
                print(f"Loop stopped at iteration {iteration} due to error: {response.error}")
                break
            
            # Evaluate condition
            should_continue = self.evaluate_condition(response)
            
            if not should_continue:
                print(f"Loop condition met at iteration {iteration}. Stopping.")
                break
            
            # Check max iterations
            if iteration >= self.max_iterations:
                print(f"Maximum iterations ({self.max_iterations}) reached. Stopping.")
                break
            
            # Prepare task for next iteration
            current_task = self._create_next_iteration_task(response, iteration)
        
        return responses
    
    def evaluate_condition(self, response: AgentResponse) -> bool:
        """
        Evaluate whether to continue the loop.
        
        Args:
            response: Response from current iteration
            
        Returns:
            True to continue loop, False to stop
        """
        try:
            return self.condition(response)
        except Exception as e:
            print(f"Error evaluating condition: {e}. Stopping loop.")
            return False
    
    def _create_next_iteration_task(self, previous_response: AgentResponse, iteration: int) -> Task:
        """
        Create task for next iteration based on previous response.
        
        Args:
            previous_response: Response from previous iteration
            iteration: Current iteration number
            
        Returns:
            Task for next iteration
        """
        return Task(
            task_id=str(uuid.uuid4()),
            description=f"Continue processing based on previous iteration result.",
            input_data={
                'previous_output': previous_response.output,
                'iteration': iteration + 1
            },
            context={
                'previous_task_id': previous_response.task_id,
                'iteration': iteration + 1
            }
        )


print("✓ LoopExecutor class implemented successfully")

### Example: Loop Agent Execution

Demonstrate an agent executing in a loop until a condition is met.

In [ ]:
# Create agent for loop execution
counter_agent = ConciergeAgent(
    name='counter',
    llm_config=llm_config
)

# Define a condition function
def should_continue(response: AgentResponse) -> bool:
    """
    Condition to continue loop.
    Stops after 3 iterations for this example.
    """
    # Check iteration count from task context
    iteration = response.output.count('iteration') if response.output else 0
    return iteration < 3

# Create initial task
loop_task = Task(
    task_id=str(uuid.uuid4()),
    description="Count and describe iteration 1. Mention 'iteration' in your response.",
    input_data={'count': 1},
    context={'iteration': 1}
)

# Create loop executor and run
loop_executor = LoopExecutor(
    agent=counter_agent,
    condition=should_continue,
    max_iterations=5
)

print(f"\n{'='*60}")
print(f"LOOP EXECUTION")
print(f"{'='*60}\n")
print(f"Executing agent in loop (max {loop_executor.max_iterations} iterations)...\n")

# Execute the loop
loop_responses = loop_executor.execute_loop(loop_task)

print(f"\n{'='*60}")
print(f"LOOP RESULTS")
print(f"{'='*60}")
print(f"Total Iterations: {len(loop_responses)}")
print(f"\nIteration Details:")
for i, response in enumerate(loop_responses, 1):
    print(f"\nIteration {i}:")
    print(f"  Status: {response.status}")
    print(f"  Execution Time: {response.execution_time:.3f}s")
    output_preview = response.output[:100] + '...' if len(response.output) > 100 else response.output
    print(f"  Output: {output_preview}")

## Tools

Implementation of tool integration layer including MCP, custom, built-in, and OpenAPI tools.

### Base Tool Interface

Define the abstract base class for all tools in the system.

In [ ]:
class Tool(ABC):
    """Base interface for all tools."""
    
    def __init__(self, name: str, description: str):
        """
        Initialize a tool.
        
        Args:
            name: Unique identifier for the tool
            description: Human-readable description of what the tool does
        """
        self.name = name
        self.description = description
        self.state: Dict[str, Any] = {}
    
    @abstractmethod
    def get_schema(self) -> Dict[str, Any]:
        """
        Get the tool's schema definition.
        
        Returns:
            Dictionary containing the tool schema with parameters and types
        """
        pass
    
    @abstractmethod
    def execute(self, params: Dict[str, Any]) -> ToolResult:
        """
        Execute the tool with given parameters.
        
        Args:
            params: Dictionary of parameters for tool execution
            
        Returns:
            ToolResult containing execution outcome
        """
        pass
    
    @abstractmethod
    def validate_params(self, params: Dict[str, Any]) -> bool:
        """
        Validate parameters before execution.
        
        Args:
            params: Dictionary of parameters to validate
            
        Returns:
            True if parameters are valid, False otherwise
        """
        pass
    
    def restore_state(self, state: Dict[str, Any]) -> None:
        """
        Restore tool state from saved state.
        
        Args:
            state: Dictionary containing saved tool state
        """
        self.state = state.copy()
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(name='{self.name}')"


print("✓ Tool base interface defined successfully")

### Tool Registration and Management

Enhance the ConciergeAgent class with improved tool management capabilities.

In [ ]:
# Custom exceptions for tool operations
class ToolExecutionError(Exception):
    """Error during tool execution."""
    pass


class ValidationError(Exception):
    """Error in input validation."""
    pass


class ToolRegistry:
    """Registry for managing tools available to agents."""
    
    def __init__(self):
        self.tools: Dict[str, Tool] = {}
        self.tool_schemas: Dict[str, Dict[str, Any]] = {}
    
    def register(self, tool: Tool) -> None:
        """
        Register a tool in the registry.
        
        Args:
            tool: Tool instance to register
            
        Raises:
            ValidationError: If tool validation fails
        """
        # Validate tool has required methods
        if not isinstance(tool, Tool):
            raise ValidationError(f"Tool must inherit from Tool base class")
        
        # Validate tool name is unique
        if tool.name in self.tools:
            raise ValidationError(f"Tool with name '{tool.name}' already registered")
        
        # Get and validate schema
        try:
            schema = tool.get_schema()
            self._validate_schema(schema)
        except Exception as e:
            raise ValidationError(f"Invalid tool schema for '{tool.name}': {str(e)}")
        
        # Register tool
        self.tools[tool.name] = tool
        self.tool_schemas[tool.name] = schema
    
    def unregister(self, tool_name: str) -> None:
        """
        Unregister a tool from the registry.
        
        Args:
            tool_name: Name of the tool to unregister
        """
        if tool_name in self.tools:
            del self.tools[tool_name]
            del self.tool_schemas[tool_name]
    
    def get_tool(self, tool_name: str) -> Optional[Tool]:
        """
        Get a tool by name.
        
        Args:
            tool_name: Name of the tool to retrieve
            
        Returns:
            Tool instance or None if not found
        """
        return self.tools.get(tool_name)
    
    def get_schema(self, tool_name: str) -> Optional[Dict[str, Any]]:
        """
        Get a tool's schema by name.
        
        Args:
            tool_name: Name of the tool
            
        Returns:
            Tool schema or None if not found
        """
        return self.tool_schemas.get(tool_name)
    
    def list_tools(self) -> List[str]:
        """
        List all registered tool names.
        
        Returns:
            List of tool names
        """
        return list(self.tools.keys())
    
    def execute_tool(self, tool_name: str, params: Dict[str, Any]) -> ToolResult:
        """
        Execute a tool with given parameters.
        
        Args:
            tool_name: Name of the tool to execute
            params: Parameters for tool execution
            
        Returns:
            ToolResult from execution
            
        Raises:
            ToolExecutionError: If tool execution fails
        """
        tool = self.get_tool(tool_name)
        if not tool:
            raise ToolExecutionError(f"Tool '{tool_name}' not found")
        
        # Validate parameters
        try:
            if not tool.validate_params(params):
                raise ValidationError(f"Invalid parameters for tool '{tool_name}'")
        except Exception as e:
            raise ValidationError(f"Parameter validation failed for '{tool_name}': {str(e)}")
        
        # Execute tool
        try:
            return tool.execute(params)
        except Exception as e:
            raise ToolExecutionError(f"Tool '{tool_name}' execution failed: {str(e)}")
    
    def _validate_schema(self, schema: Dict[str, Any]) -> None:
        """
        Validate a tool schema structure.
        
        Args:
            schema: Schema dictionary to validate
            
        Raises:
            ValidationError: If schema is invalid
        """
        required_keys = ['name', 'description', 'parameters']
        for key in required_keys:
            if key not in schema:
                raise ValidationError(f"Schema missing required key: {key}")
        
        # Validate parameters structure
        if not isinstance(schema['parameters'], dict):
            raise ValidationError("Schema 'parameters' must be a dictionary")


print("✓ ToolRegistry class implemented successfully")

### Enhanced ConciergeAgent with Tool Management

Update the ConciergeAgent to use the ToolRegistry for better tool management.

In [ ]:
# Note: This enhances the existing ConciergeAgent class with tool registry
# In practice, you would modify the original class definition
# For demonstration, we'll show the enhanced add_tool method

def enhanced_add_tool(self, tool: Tool) -> None:
    """
    Enhanced tool registration with validation.
    
    Args:
        tool: Tool instance to register
        
    Raises:
        ValidationError: If tool validation fails
    """
    if not hasattr(self, 'tool_registry'):
        self.tool_registry = ToolRegistry()
    
    # Register tool in registry
    self.tool_registry.register(tool)
    
    # Also keep in tools dict for backward compatibility
    self.tools[tool.name] = tool
    
    print(f"✓ Tool '{tool.name}' registered successfully")


def get_available_tools(self) -> List[str]:
    """
    Get list of available tool names.
    
    Returns:
        List of registered tool names
    """
    if hasattr(self, 'tool_registry'):
        return self.tool_registry.list_tools()
    return list(self.tools.keys())


def get_tool_schema(self, tool_name: str) -> Optional[Dict[str, Any]]:
    """
    Get schema for a specific tool.
    
    Args:
        tool_name: Name of the tool
        
    Returns:
        Tool schema or None if not found
    """
    if hasattr(self, 'tool_registry'):
        return self.tool_registry.get_schema(tool_name)
    
    tool = self.tools.get(tool_name)
    if tool and hasattr(tool, 'get_schema'):
        return tool.get_schema()
    return None


# Monkey-patch the ConciergeAgent class with enhanced methods
ConciergeAgent.add_tool = enhanced_add_tool
ConciergeAgent.get_available_tools = get_available_tools
ConciergeAgent.get_tool_schema = get_tool_schema

print("✓ ConciergeAgent enhanced with tool management capabilities")

### Example: Tool Registration and Validation

Demonstrate the tool registration system with a simple example tool.

In [ ]:
# Create a simple example tool for demonstration
class EchoTool(Tool):
    """Simple tool that echoes back the input."""
    
    def __init__(self):
        super().__init__(
            name='echo',
            description='Echoes back the provided message'
        )
    
    def get_schema(self) -> Dict[str, Any]:
        return {
            'name': self.name,
            'description': self.description,
            'parameters': {
                'message': {
                    'type': 'string',
                    'description': 'The message to echo back',
                    'required': True
                }
            }
        }
    
    def validate_params(self, params: Dict[str, Any]) -> bool:
        if 'message' not in params:
            return False
        if not isinstance(params['message'], str):
            return False
        return True
    
    def execute(self, params: Dict[str, Any]) -> ToolResult:
        message = params['message']
        return ToolResult(
            success=True,
            data={'echo': message},
            metadata={'tool': self.name}
        )


# Create and register the tool
echo_tool = EchoTool()
print(f"✓ Created {echo_tool}")
print(f"  Description: {echo_tool.description}")

In [ ]:
# Create a tool registry and register the tool
registry = ToolRegistry()
registry.register(echo_tool)

print(f"\n{'='*60}")
print(f"TOOL REGISTRY")
print(f"{'='*60}")
print(f"Registered Tools: {registry.list_tools()}")
print(f"\nTool Schema:")
schema = registry.get_schema('echo')
print(json.dumps(schema, indent=2))

In [ ]:
# Test tool execution
params = {'message': 'Hello, Tool System!'}
result = registry.execute_tool('echo', params)

print(f"\n{'='*60}")
print(f"TOOL EXECUTION")
print(f"{'='*60}")
print(f"Tool: echo")
print(f"Parameters: {params}")
print(f"Success: {result.success}")
print(f"Data: {result.data}")
print(f"Metadata: {result.metadata}")

In [ ]:
# Test parameter validation
print(f"\n{'='*60}")
print(f"PARAMETER VALIDATION")
print(f"{'='*60}")

# Valid parameters
valid_params = {'message': 'Test'}
print(f"Valid params {valid_params}: {echo_tool.validate_params(valid_params)}")

# Invalid parameters - missing message
invalid_params1 = {}
print(f"Invalid params {invalid_params1}: {echo_tool.validate_params(invalid_params1)}")

# Invalid parameters - wrong type
invalid_params2 = {'message': 123}
print(f"Invalid params {invalid_params2}: {echo_tool.validate_params(invalid_params2)}")

In [ ]:
# Test error handling
print(f"\n{'='*60}")
print(f"ERROR HANDLING")
print(f"{'='*60}")

# Try to execute with invalid parameters
try:
    result = registry.execute_tool('echo', {})
except ValidationError as e:
    print(f"✓ Caught ValidationError: {e}")

# Try to execute non-existent tool
try:
    result = registry.execute_tool('nonexistent', {})
except ToolExecutionError as e:
    print(f"✓ Caught ToolExecutionError: {e}")

# Try to register duplicate tool
try:
    registry.register(echo_tool)
except ValidationError as e:
    print(f"✓ Caught ValidationError for duplicate: {e}")

In [ ]:
# Demonstrate agent integration
test_agent = ConciergeAgent(
    name='tool-test-agent',
    llm_config=llm_config
)

# Add tool to agent
test_agent.add_tool(echo_tool)

print(f"\n{'='*60}")
print(f"AGENT TOOL INTEGRATION")
print(f"{'='*60}")
print(f"Agent: {test_agent.agent_id}")
print(f"Available Tools: {test_agent.get_available_tools()}")
print(f"\nTool Schema:")
schema = test_agent.get_tool_schema('echo')
if schema:
    print(json.dumps(schema, indent=2))

### Custom Tool Implementation

Implement the CustomTool class that allows users to define their own tools with custom logic.

In [ ]:
class CustomTool(Tool):
    """User-defined custom tool with flexible schema and handler."""
    
    def __init__(
        self,
        name: str,
        description: str,
        parameters: Dict[str, Any],
        handler: Callable[[Dict[str, Any]], Any]
    ):
        """
        Initialize a custom tool.
        
        Args:
            name: Unique identifier for the tool
            description: Human-readable description
            parameters: Schema definition for tool parameters
            handler: Callable function that implements the tool logic
        """
        super().__init__(name, description)
        self.parameters = parameters
        self.handler = handler
        self._validate_handler()
    
    def _validate_handler(self) -> None:
        """Validate that the handler is callable."""
        if not callable(self.handler):
            raise ValidationError(f"Handler for tool '{self.name}' must be callable")
    
    def get_schema(self) -> Dict[str, Any]:
        """
        Get the tool's schema definition.
        
        Returns:
            Dictionary containing the tool schema
        """
        return {
            'name': self.name,
            'description': self.description,
            'parameters': self.parameters
        }
    
    def validate_params(self, params: Dict[str, Any]) -> bool:
        """
        Validate parameters against the schema.
        
        Args:
            params: Parameters to validate
            
        Returns:
            True if valid, False otherwise
        """
        # Check required parameters
        for param_name, param_spec in self.parameters.items():
            if param_spec.get('required', False):
                if param_name not in params:
                    return False
        
        # Validate parameter types
        for param_name, param_value in params.items():
            if param_name in self.parameters:
                expected_type = self.parameters[param_name].get('type')
                if expected_type:
                    if not self._check_type(param_value, expected_type):
                        return False
        
        return True
    
    def _check_type(self, value: Any, expected_type: str) -> bool:
        """Check if value matches expected type."""
        type_mapping = {
            'string': str,
            'number': (int, float),
            'integer': int,
            'boolean': bool,
            'array': list,
            'object': dict
        }
        expected_python_type = type_mapping.get(expected_type)
        if expected_python_type:
            return isinstance(value, expected_python_type)
        return True
    
    def execute(self, params: Dict[str, Any]) -> ToolResult:
        """
        Execute the custom tool handler.
        
        Args:
            params: Parameters for execution
            
        Returns:
            ToolResult containing execution outcome
        """
        try:
            result = self.handler(params)
            return ToolResult(
                success=True,
                data=result,
                metadata={'tool': self.name, 'type': 'custom'}
            )
        except Exception as e:
            return ToolResult(
                success=False,
                data=None,
                error=str(e),
                metadata={'tool': self.name, 'type': 'custom'}
            )


print("✓ CustomTool class implemented successfully")

### Example Custom Tools

Create practical examples of custom tools: a calculator and a text processor.

In [ ]:
# Example 1: Calculator Tool
def calculator_handler(params: Dict[str, Any]) -> Dict[str, Any]:
    """Handler for calculator operations."""
    operation = params['operation']
    a = params['a']
    b = params['b']
    
    operations = {
        'add': lambda x, y: x + y,
        'subtract': lambda x, y: x - y,
        'multiply': lambda x, y: x * y,
        'divide': lambda x, y: x / y if y != 0 else None,
        'power': lambda x, y: x ** y,
        'modulo': lambda x, y: x % y if y != 0 else None
    }
    
    if operation not in operations:
        raise ValueError(f"Unsupported operation: {operation}")
    
    result = operations[operation](a, b)
    
    if result is None:
        raise ValueError("Division by zero")
    
    return {
        'operation': operation,
        'operands': [a, b],
        'result': result
    }


calculator_tool = CustomTool(
    name='calculator',
    description='Performs mathematical operations on two numbers',
    parameters={
        'operation': {
            'type': 'string',
            'description': 'The operation to perform: add, subtract, multiply, divide, power, modulo',
            'required': True
        },
        'a': {
            'type': 'number',
            'description': 'First operand',
            'required': True
        },
        'b': {
            'type': 'number',
            'description': 'Second operand',
            'required': True
        }
    },
    handler=calculator_handler
)

print(f"✓ Created calculator tool")
print(f"  Name: {calculator_tool.name}")
print(f"  Description: {calculator_tool.description}")

In [ ]:
# Example 2: Text Processor Tool
def text_processor_handler(params: Dict[str, Any]) -> Dict[str, Any]:
    """Handler for text processing operations."""
    text = params['text']
    operation = params['operation']
    
    operations = {
        'uppercase': lambda t: t.upper(),
        'lowercase': lambda t: t.lower(),
        'title': lambda t: t.title(),
        'reverse': lambda t: t[::-1],
        'word_count': lambda t: len(t.split()),
        'char_count': lambda t: len(t),
        'remove_spaces': lambda t: t.replace(' ', ''),
        'capitalize': lambda t: t.capitalize()
    }
    
    if operation not in operations:
        raise ValueError(f"Unsupported operation: {operation}")
    
    result = operations[operation](text)
    
    return {
        'operation': operation,
        'original': text,
        'result': result,
        'original_length': len(text)
    }


text_processor_tool = CustomTool(
    name='text_processor',
    description='Performs various text processing operations',
    parameters={
        'text': {
            'type': 'string',
            'description': 'The text to process',
            'required': True
        },
        'operation': {
            'type': 'string',
            'description': 'Operation: uppercase, lowercase, title, reverse, word_count, char_count, remove_spaces, capitalize',
            'required': True
        }
    },
    handler=text_processor_handler
)

print(f"✓ Created text processor tool")
print(f"  Name: {text_processor_tool.name}")
print(f"  Description: {text_processor_tool.description}")

### Custom Tool Demonstrations

Test the custom tools with various operations.

In [ ]:
# Test Calculator Tool
print(f"{'='*60}")
print(f"CALCULATOR TOOL TESTS")
print(f"{'='*60}\n")

test_operations = [
    {'operation': 'add', 'a': 10, 'b': 5},
    {'operation': 'subtract', 'a': 10, 'b': 5},
    {'operation': 'multiply', 'a': 10, 'b': 5},
    {'operation': 'divide', 'a': 10, 'b': 5},
    {'operation': 'power', 'a': 2, 'b': 8},
    {'operation': 'modulo', 'a': 17, 'b': 5}
]

for params in test_operations:
    result = calculator_tool.execute(params)
    if result.success:
        data = result.data
        print(f"{data['operands'][0]} {data['operation']} {data['operands'][1]} = {data['result']}")
    else:
        print(f"Error: {result.error}")

In [ ]:
# Test Text Processor Tool
print(f"\n{'='*60}")
print(f"TEXT PROCESSOR TOOL TESTS")
print(f"{'='*60}\n")

test_text = "Hello World from Custom Tools"
text_operations = ['uppercase', 'lowercase', 'title', 'reverse', 'word_count', 'char_count']

for operation in text_operations:
    params = {'text': test_text, 'operation': operation}
    result = text_processor_tool.execute(params)
    if result.success:
        data = result.data
        print(f"{operation:15} -> {data['result']}")
    else:
        print(f"{operation:15} -> Error: {result.error}")

In [ ]:
# Test parameter validation
print(f"\n{'='*60}")
print(f"PARAMETER VALIDATION TESTS")
print(f"{'='*60}\n")

# Valid parameters
valid_calc_params = {'operation': 'add', 'a': 5, 'b': 3}
print(f"Calculator valid params: {calculator_tool.validate_params(valid_calc_params)}")

# Missing required parameter
invalid_calc_params1 = {'operation': 'add', 'a': 5}
print(f"Calculator missing 'b': {calculator_tool.validate_params(invalid_calc_params1)}")

# Wrong type
invalid_calc_params2 = {'operation': 'add', 'a': 'five', 'b': 3}
print(f"Calculator wrong type: {calculator_tool.validate_params(invalid_calc_params2)}")

# Valid text processor params
valid_text_params = {'text': 'hello', 'operation': 'uppercase'}
print(f"Text processor valid: {text_processor_tool.validate_params(valid_text_params)}")

# Missing required parameter
invalid_text_params = {'text': 'hello'}
print(f"Text processor missing operation: {text_processor_tool.validate_params(invalid_text_params)}")

In [ ]:
# Test error handling in custom tools
print(f"\n{'='*60}")
print(f"ERROR HANDLING TESTS")
print(f"{'='*60}\n")

# Division by zero
result = calculator_tool.execute({'operation': 'divide', 'a': 10, 'b': 0})
print(f"Division by zero:")
print(f"  Success: {result.success}")
print(f"  Error: {result.error}")

# Invalid operation
result = calculator_tool.execute({'operation': 'invalid', 'a': 10, 'b': 5})
print(f"\nInvalid operation:")
print(f"  Success: {result.success}")
print(f"  Error: {result.error}")

# Invalid text operation
result = text_processor_tool.execute({'text': 'hello', 'operation': 'invalid'})
print(f"\nInvalid text operation:")
print(f"  Success: {result.success}")
print(f"  Error: {result.error}")

In [ ]:
# Register custom tools with an agent
custom_agent = ConciergeAgent(
    name='custom-tool-agent',
    llm_config=llm_config
)

custom_agent.add_tool(calculator_tool)
custom_agent.add_tool(text_processor_tool)

print(f"\n{'='*60}")
print(f"AGENT WITH CUSTOM TOOLS")
print(f"{'='*60}")
print(f"Agent: {custom_agent.agent_id}")
print(f"Available Tools: {custom_agent.get_available_tools()}")
print(f"\nTool Schemas:")
for tool_name in custom_agent.get_available_tools():
    schema = custom_agent.get_tool_schema(tool_name)
    print(f"\n{tool_name}:")
    print(f"  Description: {schema['description']}")
    print(f"  Parameters: {list(schema['parameters'].keys())}")

### Built-in Tools

Implementation of pre-packaged tools including Google Search and Code Execution.

In [ ]:
# Rate limiting utility
class RateLimiter:
    """Simple rate limiter for tool usage."""
    
    def __init__(self, max_calls: int, time_window: float):
        """
        Initialize rate limiter.
        
        Args:
            max_calls: Maximum number of calls allowed
            time_window: Time window in seconds
        """
        self.max_calls = max_calls
        self.time_window = time_window
        self.calls: List[float] = []
    
    def is_allowed(self) -> bool:
        """Check if a call is allowed under rate limit."""
        now = time.time()
        
        # Remove old calls outside time window
        self.calls = [call_time for call_time in self.calls if now - call_time < self.time_window]
        
        # Check if under limit
        if len(self.calls) < self.max_calls:
            self.calls.append(now)
            return True
        return False
    
    def get_wait_time(self) -> float:
        """Get time to wait before next call is allowed."""
        if not self.calls:
            return 0.0
        now = time.time()
        oldest_call = min(self.calls)
        return max(0.0, self.time_window - (now - oldest_call))


print("✓ RateLimiter utility implemented")

In [ ]:
class GoogleSearchTool(Tool):
    """Built-in Google Search integration using Custom Search API."""
    
    def __init__(self, api_key: Optional[str] = None, search_engine_id: Optional[str] = None):
        """
        Initialize Google Search tool.
        
        Args:
            api_key: Google Custom Search API key
            search_engine_id: Custom Search Engine ID
        """
        super().__init__(
            name='google_search',
            description='Search the web using Google Custom Search API'
        )
        self.api_key = api_key or os.getenv('GOOGLE_SEARCH_API_KEY')
        self.search_engine_id = search_engine_id or os.getenv('GOOGLE_SEARCH_ENGINE_ID')
        self.rate_limiter = RateLimiter(max_calls=10, time_window=60.0)  # 10 calls per minute
        self.base_url = 'https://www.googleapis.com/customsearch/v1'
    
    def get_schema(self) -> Dict[str, Any]:
        return {
            'name': self.name,
            'description': self.description,
            'parameters': {
                'query': {
                    'type': 'string',
                    'description': 'The search query',
                    'required': True
                },
                'num_results': {
                    'type': 'integer',
                    'description': 'Number of results to return (1-10)',
                    'required': False,
                    'default': 5
                }
            }
        }
    
    def validate_params(self, params: Dict[str, Any]) -> bool:
        if 'query' not in params:
            return False
        if not isinstance(params['query'], str):
            return False
        if 'num_results' in params:
            num = params['num_results']
            if not isinstance(num, int) or num < 1 or num > 10:
                return False
        return True
    
    def execute(self, params: Dict[str, Any]) -> ToolResult:
        """
        Execute Google search.
        
        Args:
            params: Search parameters
            
        Returns:
            ToolResult with search results
        """
        # Check authentication
        if not self.api_key or not self.search_engine_id:
            return ToolResult(
                success=False,
                data=None,
                error='Google Search API credentials not configured. Set GOOGLE_SEARCH_API_KEY and GOOGLE_SEARCH_ENGINE_ID.',
                metadata={'tool': self.name}
            )
        
        # Check rate limit
        if not self.rate_limiter.is_allowed():
            wait_time = self.rate_limiter.get_wait_time()
            return ToolResult(
                success=False,
                data=None,
                error=f'Rate limit exceeded. Please wait {wait_time:.1f} seconds.',
                metadata={'tool': self.name, 'wait_time': wait_time}
            )
        
        query = params['query']
        num_results = params.get('num_results', 5)
        
        try:
            # Make API request
            response = requests.get(
                self.base_url,
                params={
                    'key': self.api_key,
                    'cx': self.search_engine_id,
                    'q': query,
                    'num': num_results
                },
                timeout=10
            )
            response.raise_for_status()
            
            data = response.json()
            
            # Extract search results
            results = []
            for item in data.get('items', []):
                results.append({
                    'title': item.get('title'),
                    'link': item.get('link'),
                    'snippet': item.get('snippet'),
                    'displayLink': item.get('displayLink')
                })
            
            return ToolResult(
                success=True,
                data={
                    'query': query,
                    'results': results,
                    'total_results': len(results)
                },
                metadata={'tool': self.name, 'api': 'google_custom_search'}
            )
            
        except requests.exceptions.RequestException as e:
            return ToolResult(
                success=False,
                data=None,
                error=f'Search request failed: {str(e)}',
                metadata={'tool': self.name}
            )
        except Exception as e:
            return ToolResult(
                success=False,
                data=None,
                error=f'Unexpected error: {str(e)}',
                metadata={'tool': self.name}
            )


print("✓ GoogleSearchTool implemented successfully")

In [ ]:
import subprocess
import sys
import tempfile
import os as os_module


class CodeExecutionTool(Tool):
    """Built-in code execution in a sandboxed environment."""
    
    def __init__(self, timeout: int = 30, allowed_languages: Optional[List[str]] = None):
        """
        Initialize code execution tool.
        
        Args:
            timeout: Maximum execution time in seconds
            allowed_languages: List of allowed programming languages
        """
        super().__init__(
            name='code_execution',
            description='Execute code in a sandboxed environment'
        )
        self.timeout = timeout
        self.allowed_languages = allowed_languages or ['python']
        self.rate_limiter = RateLimiter(max_calls=20, time_window=60.0)  # 20 calls per minute
    
    def get_schema(self) -> Dict[str, Any]:
        return {
            'name': self.name,
            'description': self.description,
            'parameters': {
                'code': {
                    'type': 'string',
                    'description': 'The code to execute',
                    'required': True
                },
                'language': {
                    'type': 'string',
                    'description': f'Programming language ({', '.join(self.allowed_languages)})',
                    'required': False,
                    'default': 'python'
                }
            }
        }
    
    def validate_params(self, params: Dict[str, Any]) -> bool:
        if 'code' not in params:
            return False
        if not isinstance(params['code'], str):
            return False
        if 'language' in params:
            if params['language'] not in self.allowed_languages:
                return False
        return True
    
    def execute(self, params: Dict[str, Any]) -> ToolResult:
        """
        Execute code in sandboxed environment.
        
        Args:
            params: Execution parameters
            
        Returns:
            ToolResult with execution output
        """
        # Check rate limit
        if not self.rate_limiter.is_allowed():
            wait_time = self.rate_limiter.get_wait_time()
            return ToolResult(
                success=False,
                data=None,
                error=f'Rate limit exceeded. Please wait {wait_time:.1f} seconds.',
                metadata={'tool': self.name, 'wait_time': wait_time}
            )
        
        code = params['code']
        language = params.get('language', 'python')
        
        if language == 'python':
            return self._execute_python(code)
        else:
            return ToolResult(
                success=False,
                data=None,
                error=f'Unsupported language: {language}',
                metadata={'tool': self.name}
            )
    
    def _execute_python(self, code: str) -> ToolResult:
        """Execute Python code in a subprocess."""
        try:
            # Create temporary file for code
            with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
                f.write(code)
                temp_file = f.name
            
            try:
                # Execute in subprocess with timeout
                result = subprocess.run(
                    [sys.executable, temp_file],
                    capture_output=True,
                    text=True,
                    timeout=self.timeout
                )
                
                return ToolResult(
                    success=result.returncode == 0,
                    data={
                        'stdout': result.stdout,
                        'stderr': result.stderr,
                        'returncode': result.returncode
                    },
                    error=result.stderr if result.returncode != 0 else None,
                    metadata={'tool': self.name, 'language': 'python', 'timeout': self.timeout}
                )
            finally:
                # Clean up temporary file
                try:
                    os_module.unlink(temp_file)
                except:
                    pass
                    
        except subprocess.TimeoutExpired:
            return ToolResult(
                success=False,
                data=None,
                error=f'Code execution timed out after {self.timeout} seconds',
                metadata={'tool': self.name, 'timeout': self.timeout}
            )
        except Exception as e:
            return ToolResult(
                success=False,
                data=None,
                error=f'Execution error: {str(e)}',
                metadata={'tool': self.name}
            )


print("✓ CodeExecutionTool implemented successfully")

### Built-in Tools Demonstrations

Test the built-in tools with various examples.

In [ ]:
# Create built-in tools
google_search = GoogleSearchTool()
code_executor = CodeExecutionTool(timeout=10)

print(f"✓ Created built-in tools")
print(f"  Google Search: {google_search.name}")
print(f"  Code Execution: {code_executor.name}")

In [ ]:
# Test Google Search Tool (will show error if credentials not configured)
print(f"{'='*60}")
print(f"GOOGLE SEARCH TOOL TEST")
print(f"{'='*60}\n")

search_params = {
    'query': 'artificial intelligence multi-agent systems',
    'num_results': 3
}

result = google_search.execute(search_params)
print(f"Success: {result.success}")

if result.success:
    data = result.data
    print(f"Query: {data['query']}")
    print(f"Total Results: {data['total_results']}\n")
    for i, item in enumerate(data['results'], 1):
        print(f"{i}. {item['title']}")
        print(f"   {item['link']}")
        print(f"   {item['snippet'][:100]}...\n")
else:
    print(f"Error: {result.error}")
    print(f"\nNote: To use Google Search, configure GOOGLE_SEARCH_API_KEY and GOOGLE_SEARCH_ENGINE_ID")

In [ ]:
# Test Code Execution Tool - Simple calculation
print(f"\n{'='*60}")
print(f"CODE EXECUTION TOOL TEST 1: Simple Calculation")
print(f"{'='*60}\n")

code1 = """
# Calculate factorial
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n - 1)

result = factorial(5)
print(f"Factorial of 5 is: {result}")
"""

result = code_executor.execute({'code': code1, 'language': 'python'})
print(f"Success: {result.success}")
if result.success:
    print(f"Output:\n{result.data['stdout']}")
else:
    print(f"Error: {result.error}")

In [ ]:
# Test Code Execution Tool - Data processing
print(f"\n{'='*60}")
print(f"CODE EXECUTION TOOL TEST 2: Data Processing")
print(f"{'='*60}\n")

code2 = """
# Process a list of numbers
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Calculate statistics
total = sum(numbers)
average = total / len(numbers)
evens = [n for n in numbers if n % 2 == 0]
odds = [n for n in numbers if n % 2 != 0]

print(f"Numbers: {numbers}")
print(f"Sum: {total}")
print(f"Average: {average}")
print(f"Evens: {evens}")
print(f"Odds: {odds}")
"""

result = code_executor.execute({'code': code2})
print(f"Success: {result.success}")
if result.success:
    print(f"Output:\n{result.data['stdout']}")
else:
    print(f"Error: {result.error}")

In [ ]:
# Test Code Execution Tool - Error handling
print(f"\n{'='*60}")
print(f"CODE EXECUTION TOOL TEST 3: Error Handling")
print(f"{'='*60}\n")

code3 = """
# This code will raise an error
x = 10
y = 0
result = x / y  # Division by zero
print(result)
"""

result = code_executor.execute({'code': code3})
print(f"Success: {result.success}")
print(f"Return Code: {result.data['returncode']}")
if not result.success:
    print(f"Error Output:\n{result.data['stderr']}")

In [ ]:
# Test rate limiting
print(f"\n{'='*60}")
print(f"RATE LIMITING TEST")
print(f"{'='*60}\n")

# Create a tool with strict rate limit for testing
limited_executor = CodeExecutionTool(timeout=5)
limited_executor.rate_limiter = RateLimiter(max_calls=3, time_window=10.0)

simple_code = "print('Hello')"

for i in range(5):
    result = limited_executor.execute({'code': simple_code})
    if result.success:
        print(f"Call {i+1}: Success")
    else:
        print(f"Call {i+1}: {result.error}")

In [ ]:
# Register built-in tools with an agent
builtin_agent = ConciergeAgent(
    name='builtin-tools-agent',
    llm_config=llm_config
)

builtin_agent.add_tool(google_search)
builtin_agent.add_tool(code_executor)

print(f"\n{'='*60}")
print(f"AGENT WITH BUILT-IN TOOLS")
print(f"{'='*60}")
print(f"Agent: {builtin_agent.agent_id}")
print(f"Available Tools: {builtin_agent.get_available_tools()}")
print(f"\nTool Details:")
for tool_name in builtin_agent.get_available_tools():
    schema = builtin_agent.get_tool_schema(tool_name)
    print(f"\n{tool_name}:")
    print(f"  Description: {schema['description']}")
    print(f"  Parameters: {list(schema['parameters'].keys())}")

### MCP Tool Integration

Implementation of Model Context Protocol (MCP) tool integration for standardized tool communication.

In [ ]:
# MCP Protocol data structures
@dataclass
class MCPRequest:
    """MCP protocol request."""
    method: str
    params: Dict[str, Any]
    id: str = field(default_factory=lambda: str(uuid.uuid4()))
    jsonrpc: str = "2.0"
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            'jsonrpc': self.jsonrpc,
            'id': self.id,
            'method': self.method,
            'params': self.params
        }


@dataclass
class MCPResponse:
    """MCP protocol response."""
    id: str
    result: Optional[Any] = None
    error: Optional[Dict[str, Any]] = None
    jsonrpc: str = "2.0"
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'MCPResponse':
        return cls(
            id=data.get('id', ''),
            result=data.get('result'),
            error=data.get('error'),
            jsonrpc=data.get('jsonrpc', '2.0')
        )


print("✓ MCP protocol data structures defined")

In [ ]:
class MCPTool(Tool):
    """Model Context Protocol tool wrapper."""
    
    def __init__(self, mcp_endpoint: str, tool_name: str, tool_schema: Optional[Dict[str, Any]] = None):
        """
        Initialize MCP tool.
        
        Args:
            mcp_endpoint: URL endpoint for MCP server
            tool_name: Name of the tool in MCP server
            tool_schema: Optional pre-defined schema (will fetch if not provided)
        """
        super().__init__(
            name=tool_name,
            description=f'MCP tool: {tool_name}'
        )
        self.mcp_endpoint = mcp_endpoint
        self.tool_schema = tool_schema
        self.timeout = 30
        
        # Fetch schema if not provided
        if not self.tool_schema:
            self._fetch_schema()
    
    def _fetch_schema(self) -> None:
        """Fetch tool schema from MCP server."""
        try:
            request = MCPRequest(
                method='tools/list',
                params={}
            )
            response = self._invoke_mcp(request)
            
            if response.error:
                raise ToolExecutionError(f"Failed to fetch schema: {response.error}")
            
            # Find this tool's schema in the list
            tools = response.result.get('tools', [])
            for tool in tools:
                if tool.get('name') == self.name:
                    self.tool_schema = tool
                    self.description = tool.get('description', self.description)
                    break
            
            if not self.tool_schema:
                # Create a default schema
                self.tool_schema = {
                    'name': self.name,
                    'description': self.description,
                    'parameters': {}
                }
        except Exception as e:
            # If schema fetch fails, use default
            self.tool_schema = {
                'name': self.name,
                'description': self.description,
                'parameters': {}
            }
    
    def get_schema(self) -> Dict[str, Any]:
        """Get the tool's schema."""
        return self.tool_schema
    
    def validate_params(self, params: Dict[str, Any]) -> bool:
        """
        Validate parameters against MCP schema.
        
        Args:
            params: Parameters to validate
            
        Returns:
            True if valid, False otherwise
        """
        schema_params = self.tool_schema.get('parameters', {})
        
        # Check required parameters
        for param_name, param_spec in schema_params.items():
            if param_spec.get('required', False):
                if param_name not in params:
                    return False
        
        return True
    
    def execute(self, params: Dict[str, Any]) -> ToolResult:
        """
        Execute the MCP tool.
        
        Args:
            params: Tool parameters
            
        Returns:
            ToolResult with execution outcome
        """
        try:
            request = MCPRequest(
                method='tools/call',
                params={
                    'name': self.name,
                    'arguments': params
                }
            )
            
            response = self._invoke_mcp(request)
            
            if response.error:
                return ToolResult(
                    success=False,
                    data=None,
                    error=f"MCP error: {response.error.get('message', 'Unknown error')}",
                    metadata={'tool': self.name, 'protocol': 'mcp'}
                )
            
            return ToolResult(
                success=True,
                data=response.result,
                metadata={'tool': self.name, 'protocol': 'mcp'}
            )
            
        except Exception as e:
            return ToolResult(
                success=False,
                data=None,
                error=f"MCP invocation failed: {str(e)}",
                metadata={'tool': self.name, 'protocol': 'mcp'}
            )
    
    def _invoke_mcp(self, request: MCPRequest) -> MCPResponse:
        """
        Invoke MCP server with a request.
        
        Args:
            request: MCP request to send
            
        Returns:
            MCP response
            
        Raises:
            ToolExecutionError: If invocation fails
        """
        try:
            response = requests.post(
                self.mcp_endpoint,
                json=request.to_dict(),
                headers={'Content-Type': 'application/json'},
                timeout=self.timeout
            )
            response.raise_for_status()
            
            return MCPResponse.from_dict(response.json())
            
        except requests.exceptions.RequestException as e:
            raise ToolExecutionError(f"MCP request failed: {str(e)}")
        except Exception as e:
            raise ToolExecutionError(f"MCP invocation error: {str(e)}")


print("✓ MCPTool class implemented successfully")

### MCP Mock Server for Testing

Create a simple mock MCP server for demonstration purposes.

In [ ]:
class MockMCPServer:
    """Mock MCP server for testing and demonstration."""
    
    def __init__(self):
        self.tools = {
            'weather': {
                'name': 'weather',
                'description': 'Get weather information for a location',
                'parameters': {
                    'location': {
                        'type': 'string',
                        'description': 'City name or location',
                        'required': True
                    },
                    'units': {
                        'type': 'string',
                        'description': 'Temperature units (celsius or fahrenheit)',
                        'required': False,
                        'default': 'celsius'
                    }
                }
            },
            'translate': {
                'name': 'translate',
                'description': 'Translate text between languages',
                'parameters': {
                    'text': {
                        'type': 'string',
                        'description': 'Text to translate',
                        'required': True
                    },
                    'target_language': {
                        'type': 'string',
                        'description': 'Target language code',
                        'required': True
                    }
                }
            }
        }
    
    def handle_request(self, request_data: Dict[str, Any]) -> Dict[str, Any]:
        """Handle MCP request and return response."""
        method = request_data.get('method')
        params = request_data.get('params', {})
        request_id = request_data.get('id')
        
        if method == 'tools/list':
            return {
                'jsonrpc': '2.0',
                'id': request_id,
                'result': {
                    'tools': list(self.tools.values())
                }
            }
        
        elif method == 'tools/call':
            tool_name = params.get('name')
            arguments = params.get('arguments', {})
            
            if tool_name == 'weather':
                location = arguments.get('location')
                units = arguments.get('units', 'celsius')
                temp = 22 if units == 'celsius' else 72
                return {
                    'jsonrpc': '2.0',
                    'id': request_id,
                    'result': {
                        'location': location,
                        'temperature': temp,
                        'units': units,
                        'condition': 'Partly Cloudy',
                        'humidity': 65
                    }
                }
            
            elif tool_name == 'translate':
                text = arguments.get('text')
                target = arguments.get('target_language')
                # Mock translation
                translations = {
                    'es': f"[ES] {text}",
                    'fr': f"[FR] {text}",
                    'de': f"[DE] {text}"
                }
                return {
                    'jsonrpc': '2.0',
                    'id': request_id,
                    'result': {
                        'original': text,
                        'translated': translations.get(target, f"[{target.upper()}] {text}"),
                        'target_language': target
                    }
                }
            
            return {
                'jsonrpc': '2.0',
                'id': request_id,
                'error': {
                    'code': -32601,
                    'message': f"Tool not found: {tool_name}"
                }
            }
        
        return {
            'jsonrpc': '2.0',
            'id': request_id,
            'error': {
                'code': -32601,
                'message': f"Method not found: {method}"
            }
        }


# Create mock server instance
mock_mcp_server = MockMCPServer()
print("✓ Mock MCP server created")

In [ ]:
# Create a mock MCP client that uses the mock server
class MockMCPTool(MCPTool):
    """MCP tool that uses mock server for testing."""
    
    def __init__(self, tool_name: str, mock_server: MockMCPServer):
        self.mock_server = mock_server
        # Initialize with mock endpoint
        super().__init__(
            mcp_endpoint='mock://localhost',
            tool_name=tool_name
        )
    
    def _invoke_mcp(self, request: MCPRequest) -> MCPResponse:
        """Override to use mock server instead of HTTP."""
        response_data = self.mock_server.handle_request(request.to_dict())
        return MCPResponse.from_dict(response_data)


print("✓ Mock MCP tool client created")

### MCP Tool Demonstrations

Test MCP tools with the mock server.

In [ ]:
# Create MCP tools using mock server
weather_tool = MockMCPTool('weather', mock_mcp_server)
translate_tool = MockMCPTool('translate', mock_mcp_server)

print(f"{'='*60}")
print(f"MCP TOOLS CREATED")
print(f"{'='*60}\n")
print(f"Weather Tool: {weather_tool.name}")
print(f"  Description: {weather_tool.description}")
print(f"\nTranslate Tool: {translate_tool.name}")
print(f"  Description: {translate_tool.description}")

In [ ]:
# Test weather tool
print(f"\n{'='*60}")
print(f"MCP WEATHER TOOL TEST")
print(f"{'='*60}\n")

weather_params = {
    'location': 'San Francisco',
    'units': 'celsius'
}

result = weather_tool.execute(weather_params)
print(f"Success: {result.success}")
if result.success:
    data = result.data
    print(f"\nWeather in {data['location']}:")
    print(f"  Temperature: {data['temperature']}°{data['units'][0].upper()}")
    print(f"  Condition: {data['condition']}")
    print(f"  Humidity: {data['humidity']}%")
else:
    print(f"Error: {result.error}")

In [ ]:
# Test translate tool
print(f"\n{'='*60}")
print(f"MCP TRANSLATE TOOL TEST")
print(f"{'='*60}\n")

translate_params = {
    'text': 'Hello, how are you?',
    'target_language': 'es'
}

result = translate_tool.execute(translate_params)
print(f"Success: {result.success}")
if result.success:
    data = result.data
    print(f"\nOriginal: {data['original']}")
    print(f"Translated ({data['target_language']}): {data['translated']}")
else:
    print(f"Error: {result.error}")

In [ ]:
# Test schema validation
print(f"\n{'='*60}")
print(f"MCP SCHEMA VALIDATION TEST")
print(f"{'='*60}\n")

# Display weather tool schema
weather_schema = weather_tool.get_schema()
print("Weather Tool Schema:")
print(json.dumps(weather_schema, indent=2))

# Test parameter validation
valid_params = {'location': 'New York', 'units': 'fahrenheit'}
invalid_params = {'units': 'celsius'}  # Missing required 'location'

print(f"\nValidation Results:")
print(f"Valid params {valid_params}: {weather_tool.validate_params(valid_params)}")
print(f"Invalid params {invalid_params}: {weather_tool.validate_params(invalid_params)}")

In [ ]:
# Register MCP tools with an agent
mcp_agent = ConciergeAgent(
    name='mcp-tools-agent',
    llm_config=llm_config
)

mcp_agent.add_tool(weather_tool)
mcp_agent.add_tool(translate_tool)

print(f"\n{'='*60}")
print(f"AGENT WITH MCP TOOLS")
print(f"{'='*60}")
print(f"Agent: {mcp_agent.agent_id}")
print(f"Available Tools: {mcp_agent.get_available_tools()}")
print(f"\nTool Details:")
for tool_name in mcp_agent.get_available_tools():
    schema = mcp_agent.get_tool_schema(tool_name)
    print(f"\n{tool_name}:")
    print(f"  Description: {schema['description']}")
    print(f"  Parameters: {list(schema['parameters'].keys())}")
    print(f"  Protocol: MCP")

### OpenAPI Tool Integration

Implementation of tools generated from OpenAPI specifications for REST API integration.

In [ ]:
from urllib.parse import urljoin
import yaml


class OpenAPITool(Tool):
    """Tool generated from OpenAPI specification."""
    
    def __init__(self, spec_url: str, operation_id: str, auth: Optional[Dict[str, Any]] = None):
        """
        Initialize OpenAPI tool.
        
        Args:
            spec_url: URL or path to OpenAPI specification
            operation_id: The operation ID from the OpenAPI spec
            auth: Authentication configuration (api_key, oauth, etc.)
        """
        self.spec_url = spec_url
        self.operation_id = operation_id
        self.auth = auth or {}
        self.spec = None
        self.operation = None
        self.base_url = None
        
        # Parse OpenAPI spec
        self._parse_spec()
        
        # Initialize with parsed info
        super().__init__(
            name=operation_id,
            description=self.operation.get('summary', f'OpenAPI operation: {operation_id}')
        )
    
    def _parse_spec(self) -> None:
        """Parse OpenAPI specification."""
        try:
            # For this demo, we'll use a mock spec structure
            # In production, you would fetch and parse the actual spec
            if self.spec_url.startswith('mock://'):
                self._load_mock_spec()
            else:
                # Load real spec (simplified for demo)
                response = requests.get(self.spec_url, timeout=10)
                response.raise_for_status()
                
                if self.spec_url.endswith('.yaml') or self.spec_url.endswith('.yml'):
                    self.spec = yaml.safe_load(response.text)
                else:
                    self.spec = response.json()
                
                self._extract_operation()
        except Exception as e:
            raise ValidationError(f"Failed to parse OpenAPI spec: {str(e)}")
    
    def _load_mock_spec(self) -> None:
        """Load mock OpenAPI spec for testing."""
        self.spec = {
            'openapi': '3.0.0',
            'info': {'title': 'Mock API', 'version': '1.0.0'},
            'servers': [{'url': 'https://api.example.com'}],
            'paths': {
                '/users/{userId}': {
                    'get': {
                        'operationId': 'getUser',
                        'summary': 'Get user by ID',
                        'parameters': [
                            {
                                'name': 'userId',
                                'in': 'path',
                                'required': True,
                                'schema': {'type': 'string'}
                            }
                        ]
                    }
                },
                '/posts': {
                    'get': {
                        'operationId': 'listPosts',
                        'summary': 'List all posts',
                        'parameters': [
                            {
                                'name': 'limit',
                                'in': 'query',
                                'required': False,
                                'schema': {'type': 'integer', 'default': 10}
                            }
                        ]
                    },
                    'post': {
                        'operationId': 'createPost',
                        'summary': 'Create a new post',
                        'requestBody': {
                            'required': True,
                            'content': {
                                'application/json': {
                                    'schema': {
                                        'type': 'object',
                                        'properties': {
                                            'title': {'type': 'string'},
                                            'content': {'type': 'string'}
                                        },
                                        'required': ['title', 'content']
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
        self._extract_operation()
    
    def _extract_operation(self) -> None:
        """Extract operation details from spec."""
        # Get base URL
        servers = self.spec.get('servers', [])
        self.base_url = servers[0]['url'] if servers else 'https://api.example.com'
        
        # Find operation by operationId
        for path, path_item in self.spec.get('paths', {}).items():
            for method, operation in path_item.items():
                if isinstance(operation, dict) and operation.get('operationId') == self.operation_id:
                    self.operation = operation
                    self.operation['method'] = method.upper()
                    self.operation['path'] = path
                    return
        
        raise ValidationError(f"Operation '{self.operation_id}' not found in OpenAPI spec")
    
    def get_schema(self) -> Dict[str, Any]:
        """Generate tool schema from OpenAPI operation."""
        parameters = {}
        
        # Extract parameters from OpenAPI spec
        for param in self.operation.get('parameters', []):
            param_name = param['name']
            param_schema = param.get('schema', {})
            parameters[param_name] = {
                'type': param_schema.get('type', 'string'),
                'description': param.get('description', ''),
                'required': param.get('required', False),
                'in': param.get('in', 'query')
            }
            if 'default' in param_schema:
                parameters[param_name]['default'] = param_schema['default']
        
        # Extract request body parameters
        request_body = self.operation.get('requestBody', {})
        if request_body:
            content = request_body.get('content', {})
            json_content = content.get('application/json', {})
            schema = json_content.get('schema', {})
            properties = schema.get('properties', {})
            required_fields = schema.get('required', [])
            
            for prop_name, prop_schema in properties.items():
                parameters[prop_name] = {
                    'type': prop_schema.get('type', 'string'),
                    'description': prop_schema.get('description', ''),
                    'required': prop_name in required_fields,
                    'in': 'body'
                }
        
        return {
            'name': self.name,
            'description': self.description,
            'parameters': parameters
        }
    
    def validate_params(self, params: Dict[str, Any]) -> bool:
        """Validate parameters against OpenAPI schema."""
        schema = self.get_schema()
        
        # Check required parameters
        for param_name, param_spec in schema['parameters'].items():
            if param_spec.get('required', False):
                if param_name not in params:
                    return False
        
        return True
    
    def execute(self, params: Dict[str, Any]) -> ToolResult:
        """Execute the OpenAPI operation."""
        try:
            # Build request
            method = self.operation['method']
            path = self.operation['path']
            
            # Separate parameters by location
            path_params = {}
            query_params = {}
            body_params = {}
            
            schema = self.get_schema()
            for param_name, param_value in params.items():
                if param_name in schema['parameters']:
                    param_in = schema['parameters'][param_name].get('in', 'query')
                    if param_in == 'path':
                        path_params[param_name] = param_value
                    elif param_in == 'query':
                        query_params[param_name] = param_value
                    elif param_in == 'body':
                        body_params[param_name] = param_value
            
            # Build URL with path parameters
            url_path = path
            for param_name, param_value in path_params.items():
                url_path = url_path.replace(f'{{{param_name}}}', str(param_value))
            
            url = urljoin(self.base_url, url_path)
            
            # Make HTTP request
            response = self._make_http_request(
                url=url,
                method=method,
                params=query_params,
                json=body_params if body_params else None
            )
            
            return ToolResult(
                success=True,
                data=response,
                metadata={
                    'tool': self.name,
                    'protocol': 'openapi',
                    'method': method,
                    'url': url
                }
            )
            
        except Exception as e:
            return ToolResult(
                success=False,
                data=None,
                error=f"OpenAPI request failed: {str(e)}",
                metadata={'tool': self.name, 'protocol': 'openapi'}
            )
    
    def _make_http_request(self, url: str, method: str, **kwargs) -> Any:
        """Make HTTP request with authentication."""
        headers = kwargs.pop('headers', {})
        
        # Add authentication
        if self.auth.get('type') == 'api_key':
            key_name = self.auth.get('name', 'X-API-Key')
            key_value = self.auth.get('value')
            if self.auth.get('in') == 'header':
                headers[key_name] = key_value
            elif self.auth.get('in') == 'query':
                kwargs.setdefault('params', {})[key_name] = key_value
        
        elif self.auth.get('type') == 'bearer':
            token = self.auth.get('token')
            headers['Authorization'] = f'Bearer {token}'
        
        # For mock URLs, return mock data
        if url.startswith('https://api.example.com'):
            return self._mock_http_response(url, method, **kwargs)
        
        # Make real request
        response = requests.request(
            method=method,
            url=url,
            headers=headers,
            timeout=30,
            **kwargs
        )
        response.raise_for_status()
        
        try:
            return response.json()
        except:
            return response.text
    
    def _mock_http_response(self, url: str, method: str, **kwargs) -> Dict[str, Any]:
        """Generate mock HTTP response for testing."""
        if 'users' in url:
            return {
                'id': '123',
                'name': 'John Doe',
                'email': 'john@example.com'
            }
        elif 'posts' in url and method == 'GET':
            limit = kwargs.get('params', {}).get('limit', 10)
            return {
                'posts': [
                    {'id': i, 'title': f'Post {i}', 'content': f'Content {i}'}
                    for i in range(1, min(limit + 1, 6))
                ],
                'total': limit
            }
        elif 'posts' in url and method == 'POST':
            body = kwargs.get('json', {})
            return {
                'id': '456',
                'title': body.get('title'),
                'content': body.get('content'),
                'created_at': '2024-01-01T00:00:00Z'
            }
        return {}


print("✓ OpenAPITool class implemented successfully")

### OpenAPI Tool Demonstrations

Test OpenAPI tools with mock specifications.

In [ ]:
# Create OpenAPI tools from mock spec
get_user_tool = OpenAPITool(
    spec_url='mock://api.example.com/openapi.json',
    operation_id='getUser'
)

list_posts_tool = OpenAPITool(
    spec_url='mock://api.example.com/openapi.json',
    operation_id='listPosts'
)

create_post_tool = OpenAPITool(
    spec_url='mock://api.example.com/openapi.json',
    operation_id='createPost',
    auth={'type': 'api_key', 'name': 'X-API-Key', 'value': 'test-key', 'in': 'header'}
)

print(f"{'='*60}")
print(f"OPENAPI TOOLS CREATED")
print(f"{'='*60}\n")
print(f"Get User Tool: {get_user_tool.name}")
print(f"  Description: {get_user_tool.description}")
print(f"\nList Posts Tool: {list_posts_tool.name}")
print(f"  Description: {list_posts_tool.description}")
print(f"\nCreate Post Tool: {create_post_tool.name}")
print(f"  Description: {create_post_tool.description}")

In [ ]:
# Test Get User tool
print(f"\n{'='*60}")
print(f"OPENAPI GET USER TEST")
print(f"{'='*60}\n")

user_params = {'userId': '123'}
result = get_user_tool.execute(user_params)

print(f"Success: {result.success}")
if result.success:
    data = result.data
    print(f"\nUser Data:")
    print(f"  ID: {data['id']}")
    print(f"  Name: {data['name']}")
    print(f"  Email: {data['email']}")
else:
    print(f"Error: {result.error}")

In [ ]:
# Test List Posts tool
print(f"\n{'='*60}")
print(f"OPENAPI LIST POSTS TEST")
print(f"{'='*60}\n")

posts_params = {'limit': 3}
result = list_posts_tool.execute(posts_params)

print(f"Success: {result.success}")
if result.success:
    data = result.data
    print(f"\nPosts (Total: {data['total']}):")
    for post in data['posts']:
        print(f"  {post['id']}. {post['title']}")
else:
    print(f"Error: {result.error}")

In [ ]:
# Test Create Post tool
print(f"\n{'='*60}")
print(f"OPENAPI CREATE POST TEST")
print(f"{'='*60}\n")

create_params = {
    'title': 'My New Post',
    'content': 'This is the content of my new post created via OpenAPI tool.'
}
result = create_post_tool.execute(create_params)

print(f"Success: {result.success}")
if result.success:
    data = result.data
    print(f"\nCreated Post:")
    print(f"  ID: {data['id']}")
    print(f"  Title: {data['title']}")
    print(f"  Content: {data['content']}")
    print(f"  Created: {data['created_at']}")
else:
    print(f"Error: {result.error}")

In [ ]:
# Display OpenAPI tool schemas
print(f"\n{'='*60}")
print(f"OPENAPI TOOL SCHEMAS")
print(f"{'='*60}\n")

for tool in [get_user_tool, list_posts_tool, create_post_tool]:
    schema = tool.get_schema()
    print(f"{tool.name}:")
    print(json.dumps(schema, indent=2))
    print()

In [ ]:
# Test parameter validation
print(f"\n{'='*60}")
print(f"OPENAPI PARAMETER VALIDATION")
print(f"{'='*60}\n")

# Valid parameters
valid_user_params = {'userId': '123'}
print(f"Get User valid params: {get_user_tool.validate_params(valid_user_params)}")

# Missing required parameter
invalid_user_params = {}
print(f"Get User missing userId: {get_user_tool.validate_params(invalid_user_params)}")

# Valid create post params
valid_create_params = {'title': 'Test', 'content': 'Content'}
print(f"Create Post valid params: {create_post_tool.validate_params(valid_create_params)}")

# Missing required fields
invalid_create_params = {'title': 'Test'}
print(f"Create Post missing content: {create_post_tool.validate_params(invalid_create_params)}")

In [ ]:
# Register OpenAPI tools with an agent
openapi_agent = ConciergeAgent(
    name='openapi-tools-agent',
    llm_config=llm_config
)

openapi_agent.add_tool(get_user_tool)
openapi_agent.add_tool(list_posts_tool)
openapi_agent.add_tool(create_post_tool)

print(f"\n{'='*60}")
print(f"AGENT WITH OPENAPI TOOLS")
print(f"{'='*60}")
print(f"Agent: {openapi_agent.agent_id}")
print(f"Available Tools: {openapi_agent.get_available_tools()}")
print(f"\nTool Details:")
for tool_name in openapi_agent.get_available_tools():
    schema = openapi_agent.get_tool_schema(tool_name)
    print(f"\n{tool_name}:")
    print(f"  Description: {schema['description']}")
    print(f"  Parameters: {list(schema['parameters'].keys())}")
    print(f"  Protocol: OpenAPI")

### Tool Integration Summary

Demonstrate all tool types working together.

In [ ]:
# Create an agent with all tool types
multi_tool_agent = ConciergeAgent(
    name='multi-tool-agent',
    llm_config=llm_config
)

# Add custom tools
multi_tool_agent.add_tool(calculator_tool)
multi_tool_agent.add_tool(text_processor_tool)

# Add built-in tools
multi_tool_agent.add_tool(code_executor)

# Add MCP tools
multi_tool_agent.add_tool(weather_tool)

# Add OpenAPI tools
multi_tool_agent.add_tool(get_user_tool)

print(f"{'='*60}")
print(f"MULTI-TOOL AGENT SUMMARY")
print(f"{'='*60}")
print(f"Agent: {multi_tool_agent.agent_id}")
print(f"Total Tools: {len(multi_tool_agent.get_available_tools())}")
print(f"\nTool Inventory:")

tool_types = {
    'Custom': ['calculator', 'text_processor'],
    'Built-in': ['code_execution'],
    'MCP': ['weather'],
    'OpenAPI': ['getUser']
}

for tool_type, tools in tool_types.items():
    print(f"\n{tool_type} Tools:")
    for tool_name in tools:
        if tool_name in multi_tool_agent.get_available_tools():
            schema = multi_tool_agent.get_tool_schema(tool_name)
            print(f"  - {tool_name}: {schema['description']}")

## Advanced Features

Implementation of state management, long-running operations, session management, memory bank, and context compaction.

## Observability

Implementation of logging, tracing, and metrics collection.

## Evaluation & A2A

Implementation of agent evaluation framework and Agent-to-Agent protocol.

## Deployment

Implementation of deployment configuration and health checks.

## Examples

Complete workflow demonstrations and real-world use cases.